<H1>Raw Data Processing</H1>

In [ ]:
import os
import pandas as pd
import numpy as np
from src.feature_extraction import *


filepath = 'data/PreAttentiveGaze'
block = "Block3.tsv"
for task in ["MultipleVC", "SingleVC"]:
    whole_dataframe = pd.DataFrame()
    session_list = os.listdir(filepath)
    for session in session_list:
        participant_list = os.listdir(os.path.join(filepath, session))
        for participant in participant_list:
            stimuliIndex_list = os.listdir(os.path.join(filepath, session, participant, task))
            for stimuliIndex in stimuliIndex_list:
                dataName = f"{session}_{participant}_{task}_{stimuliIndex}_{block}"
                data = pd.read_csv(os.path.join(filepath, session, participant, task, stimuliIndex, dataName), sep='\t')
                data = fill_nan(data)
                
                this_dataframe = pd.DataFrame()
                x_data, y_data = get_gazeXY(data)
                eye_type_data = get_eye_type(data)
                velocity_data = velocity(data)
                angle_data = angular(data)

                path_length_data = get_path_length(data)
                rt_data = reaction_time(data)
                fixation_duration_data = Fixation_Duration(data)
                fixation_dispersion_data = Fixation_Dispersion(data)
                fixation_count_data = Fixation_Count(data)
                
                saccade_duration_data = Saccade_Duration(data)
                saccade_dispersion_data = Saccade_Dispersion(data)
                saccade_velocity_data, saccade_amplitude_data = Saccade_Velocity_Amplitude(data)
                saccade_count_data = Saccade_Count(data)

                mfcc_data = MFCC(velocity_data)
                left_diameter_data = pupilLeft(data)
                right_diameter_data = pupilRight(data)
                filtered_diameter_data = pupil(data)
                
                this_dataframe['x'] = [x_data]
                this_dataframe['y'] = [y_data]
                this_dataframe['eye_type'] = [eye_type_data]
                this_dataframe['velocity'] = [velocity_data]
                this_dataframe['angle'] = [angle_data]
                this_dataframe['path_length'] = [path_length_data]
                this_dataframe['reaction_time'] = [rt_data]
                this_dataframe['fixation_duration'] = [fixation_duration_data]
                this_dataframe['fixation_dispersion'] = [fixation_dispersion_data]
                this_dataframe['fixation_count'] = [fixation_count_data]
                this_dataframe['saccade_duration'] = [saccade_duration_data]
                this_dataframe['saccade_dispersion'] = [saccade_dispersion_data]
                this_dataframe['saccade_velocity'] = [saccade_velocity_data]
                this_dataframe['saccade_amplitude'] = [saccade_amplitude_data]
                this_dataframe['saccade_count'] = [saccade_count_data]
                this_dataframe['mfcc'] = [mfcc_data]
                this_dataframe['left_pupil'] = [left_diameter_data]
                this_dataframe['right_pupil'] = [right_diameter_data]
                this_dataframe['filtered_pupil'] = [filtered_diameter_data]
                this_dataframe['participant'] = [int(participant.split('_')[1])]
                this_dataframe['session'] = [int(session.split('_')[1])]
                this_dataframe['stimuli_index'] = [int(stimuliIndex.split('_')[1])]
                whole_dataframe = pd.concat([whole_dataframe, this_dataframe], ignore_index=True)

    whole_dataframe.to_csv(f"data/{task}.tsv", sep='\t', index=False)

<h1>Model</h1>

In [ ]:
import json
import pandas as pd
from src.load_data import load_data
from src.preprocessing import data_preprocessing
from src.model import Model

stimuli = "MultipleVC"
data = load_data(f"{stimuli}.tsv")
data = data_preprocessing(data)
model = Model(data)
stack = 1
results = model.leave_one_session_out_cross_validation(stack, 'ZR', 'DT', 'KNN', 'NB', 'SVM', 'LR', 'RF')
results

<H1> Sample Aggregation Results of SVM

In [ ]:
import json
import pandas as pd
from src.load_data import load_data
from src.preprocessing import data_preprocessing
from src.model import Model


total_df = pd.DataFrame()
stimuli_list = ["MultipleVC", 'SingleVC']
for stimuli in stimuli_list:
    for stack in range(1,6):
        data = load_data(f"{stimuli}.tsv")
        data = data_preprocessing(data)
        model = Model(data)
        results = model.leave_one_session_out_cross_validation(stack, 'SVM')
        acc_list = []
        for fold in range(1,6):
            accuracy = results['SVM'][str(fold)]['accuracy'][0]
            this_df = pd.DataFrame({'Stack': stack, 'Accuracy': accuracy, 'Stimuli': stimuli, 'Fold': fold}, index=[0])
            total_df = pd.concat([total_df, this_df], ignore_index=True)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = total_df.copy()

# Compute the mean accuracies
mean_accuracies = df.groupby(["Stimuli", "Stack"])["Accuracy"].mean().reset_index()

# Set the order of the "Stimuli" and "Stack" categories
order_stimuli = ["MultipleVC","SingleVC"]
order_stack = df["Stack"].unique().tolist()

# Adjust the x-values for the SVC line plot
svc_mean = mean_accuracies[mean_accuracies['Stimuli'] == 'SingleVC'].copy()
svc_mean['Adjusted_Stack'] = svc_mean['Stack']-0.8


mvc_mean = mean_accuracies[mean_accuracies['Stimuli'] == 'MultipleVC'].copy()
mvc_mean['Adjusted_Stack'] = mvc_mean['Stack']-1.2



# Define custom colors for the boxplots
custom_palette = {
    "SingleVC":"#749BC3", 
    "MultipleVC": "#A9D18E",  # RGB: 169, 209, 142
}

custom_palette_line = {
    "SingleVC":"#749BC3", 
    "MultipleVC": "#A9D18E",  # RGB: 169, 209, 142
}

totalMean = pd.concat([mvc_mean,svc_mean])

# Plotting the boxplot
plt.figure(figsize=(6, 5))

ax = sns.lineplot(x="Adjusted_Stack", y="Accuracy", hue="Stimuli", data=totalMean, hue_order=order_stimuli,palette=custom_palette_line, linewidth=1,legend=False, zorder=0)
sns.boxplot(x="Stack", y="Accuracy", hue="Stimuli", data=df, hue_order=order_stimuli, palette=custom_palette, showmeans=True, meanprops={"marker":"x",
                                                                                                                                        "markerfacecolor":"white", 
                                                                                                                                        "markeredgecolor":"black",
                                                                                                                                        "markersize":"5"}, linewidth=1,linecolor="black", zorder=3, fliersize=3)

plt.title("Accuracy by Stimuli according to Stack", fontsize=15)
plt.ylabel('Accuracy', fontsize=13)
plt.xlabel('Stack', fontsize=13)
plt.tight_layout()
plt.legend(loc='lower right')
plt.show()
